# Deep Learning Time Series COVID-19 Cases Prediction

### Import libraries and packages

In [1]:
import numpy as np
import pandas as pd
import scipy
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import requests
import pandas_profiling
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

## Data Sourcing

### Data API 

#### By country over time

In [ ]:
def fetch_time_series(feature='stringency', start_date='2020-02-14', end_date='2021-02-14'):
    """
    Get stringency time series for each countries requesting API.
    Returns json dict with TS between start_date and end_date like 'YYYY-MM-DD'.
    """
    url = f'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/{feature}/date-range/{start_date}/{end_date}'
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    data = response.json()
    return data

In [ ]:
countries_time_series_api = fetch_time_series()

In [ ]:
[(k, [c for c in v if c == 'VNM'])  for k, v in countries_time_series_api.items()  if k == 'countries' ]

In [ ]:
[([([([(vee)  for kaaa, veee  in vee.items() if kaaa in ['date_value', 'confirmed']  ])  for kaa, vee  in ve.items() if kaa =='VNM'   ])  for ka, ve  in v.items() ])  for k, v in countries_time_series_api.items() if k=='data'   ]

In [ ]:
[(k, [(ka, [(kaa, vee)  for kaa, vee  in ve.items() if kaa =='USA'   ])  for ka, ve  in v.items() ])  for k, v in countries_time_series_api.items() if k=='data'   ]

#### Country data for a specific day

In [ ]:
def fetch_data(country='USA', date='2020-08-14'):
    """
    Get stringency data for one country {ALPHA-3} requesting API.
    Returns json dict with data for country like 'AAA' and specific date and like 'YYYY-MM-DD'.
    """
    url = f'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/actions/{country}/{date}'
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    data = response.json()
    return data

In [ ]:
country_data_api = fetch_data()

In [ ]:
[[';'.join([str(kk) for kk, vv in d.items()]) for i, d in enumerate(v) if type(d) == dict and i == 0] for v in country_data_api.values()][0]

In [ ]:
[[';'.join([str(vv) for kk, vv in d.items()]) for d in v if type(d) == dict] for v in country_data_api.values()][0]

In [ ]:
# [';'.join([str(vv) for vv in v]) for v in country_data_api.values()][-1]
[';'.join([str(kk) for kk in v]) for k, v in country_data_api.items()][-1]

### CSV data

#### **Read URL**, **Get CSV files** and **store CSV in local**  *(optional do it at begining or to refresh CSV data)*

##### **get_database_to_csv()** function

In [3]:
def get_database_to_csv(url, csv_list, path='', db_grid=[]) -> list:
    """
    function that take in parameter:
     - a root URL (string) to get the CSV data,
     - a list of CSV files,
     - a path (string) to store CSV in local,
     - a grid (list of list) to add in the CSV filename, URL, local path.     
    and returns the gird updated with the CSVs of the list
    
    """

    ### Create a database grid (list of list) with all CSVs and associated URLs
    # print('db_grid', db_grid)
    #### Data project directory (if empty do not store CSV in local)
    # print('path', path)
    ### Website CSV datasets URL
    # print('url', url)
    #### List of CSVs of Website to retrieve
    # print('csv_list', csv_list)

    #### Length of grid aka number of CSVs already stored in grid
    len_grid = len(db_grid)

    for l in range(len(csv_list)):
        # g = l + len_grid
        sub_list = []       
        sub_list.append(csv_list[l]) ## 1st pos°: CSV filename
        sub_list.append(url + csv_list[l]) ## 2nd pos°: URL + CSV
        if len(data_dir) > 0: ## store CSV in local
            sub_list.append(data_dir + csv_list[l]) ## 3rd pos°: local data path + CSV
            !curl -L "{url + csv_list[l]}" > {data_dir + csv_list[l]} ## curl <URL>/<CSV> => <path>
        print('sub_list', sub_list)
        db_grid.append(sub_list)

    ### Return a database grid (list of list) with all CSVs and associated URLs
    return db_grid

#### **Get database to csv**

In [4]:
#### Data project directory
data_dir = '../raw_data/'

### Oxford Master data time series URL
url_root_oxford = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/'

#### List of CSVs of Oxford database Feel free to add more feature...
csv_list = ['confirmed_cases.csv', 'stringency_index_avg.csv', 'government_response_index_avg.csv', 
            'containment_health_index_avg.csv', 'economic_support_index.csv', 'h7_vaccination_policy.csv', 
            'c1m_school_closing.csv', 'c2m_workplace_closing.csv', 'c3m_cancel_public_events.csv', 
            'c4m_restrictions_on_gatherings.csv', 'c6m_stay_at_home_requirements.csv', 'h6m_facial_coverings.csv'
           ]
# print(csv_list)
    
### Vacinations Dataset URLs
url_root_vaccinations = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/'

#### List of CSVs of Vaccinations database
csv_list_vax = ['vaccinations.csv', 'vaccinations-by-age-group.csv']
# print(csv_list_vax)

### Create a database grid (list of list) with all CSVs and associated URLs
# db_grid = [[]]
# print(db_grid)
### Insert into database grid all CSVs and associated URLs from Oxford website
db_grid = get_database_to_csv(url_root_oxford, csv_list, data_dir) ## uncomment to store CSV
### Insert into database grid all CSVs and associated URLs from vaccinations website
db_grid = get_database_to_csv(url_root_vaccinations, csv_list_vax, data_dir, db_grid)
# print('db_grid', db_grid)

# Stack all csl in the list
csv_list += csv_list_vax

# transform list into dict:
csv = dict(zip(csv_list, [v[0] for v in enumerate(csv_list)])) # if v[1] == 'containment_health_index_avg.csv'
csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1994k  100 1994k    0     0  6329k      0 --:--:-- --:--:-- --:--:-- 6455k
sub_list ['confirmed_cases.csv', 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/confirmed_cases.csv', '../raw_data/confirmed_cases.csv']
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1888k  100 1888k    0     0  5881k      0 --:--:-- --:--:-- --:--:-- 6013k
sub_list ['stringency_index_avg.csv', 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/timeseries/stringency_index_avg.csv', '../raw_data/stringency_index_avg.csv']
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
1

{'confirmed_cases.csv': 0,
 'stringency_index_avg.csv': 1,
 'government_response_index_avg.csv': 2,
 'containment_health_index_avg.csv': 3,
 'economic_support_index.csv': 4,
 'h7_vaccination_policy.csv': 5,
 'c1m_school_closing.csv': 6,
 'c2m_workplace_closing.csv': 7,
 'c3m_cancel_public_events.csv': 8,
 'c4m_restrictions_on_gatherings.csv': 9,
 'c6m_stay_at_home_requirements.csv': 10,
 'h6m_facial_coverings.csv': 11,
 'vaccinations.csv': 12,
 'vaccinations-by-age-group.csv': 13}

### *Get database to csv one by one*


#### Target URLs
url_cases = url_root_oxford + 'confirmed_cases.csv'
url_deaths = url_root_oxford + 'confirmed_deaths.csv'

#### Index URLs
url_id1= url_root_oxford + csv_i1 ## index_strigency
url_id2 = url_root_oxford + csv_i2 ## index_gov_resp
url_id3_health = url_root_oxford + csv_i3 ## index_health
url_id4 = url_root_oxford + csv_i4 ## index_economic

#### C[1-8] URLs
url_c1 = url_root_oxford + csv_c1 ## 'C1;School closing;'
url_c2 = url_root_oxford + csv_c1 ## 'C2;Workplace closing;'
url_c3 = url_root_oxford + csv_c3 ## 'C3;Cancel public events;'
url_c4 = url_root_oxford + csv_c4 ## 'C4;Restrictions on gatherings;'
url_c5 = url_root_oxford + csv_c5 ## 'C5;Close public transport'
url_c6 = url_root_oxford + csv_c6 ## 'C6;Stay at home requirements;'
url_c7 = url_root_oxford + csv_c7 ## 'C7;Restrictions on internal movement;'
url_c8 = url_root_oxford + csv_c8 ##  'C8;International travel controls;'
 
#### E[1-4] URLs
url_e1 = url_root_oxford + csv_e1 ## 'E1;Income support;'
url_e2 = url_root_oxford + csv_e2 ## 'E2;Debt/contract relief;'
url_e3 = url_root_oxford + csv_e3 ## 'E3;Fiscal measures;'
url_e4 = url_root_oxford + csv_e4 ## 'E4;International support;'
 
#### H[1-8] URLs
url_h1 = url_root_oxford + csv_h1 ## 'H1;Public information campaigns;'
url_h2 = url_root_oxford + csv_h2 ## 'H2;Testing policy'
url_h3 = url_root_oxford + csv_h3 ## 'H3;Contact tracing;'
url_h3 = url_root_oxford + csv_h4 ## 'H4;Emergency investment in healthcare;0;0;None;None;None;USD Value',
url_h3 = url_root_oxford + csv_h5 ## 'H5;Investment in vaccines;0;0;None;None;None;USD Value',
url_h3 = url_root_oxford + csv_h6 ## 'H6;Facial Coverings;0;0;None;None;None;General;No policy',
url_h3 = url_root_oxford + csv_h7 ## 'H7;Vaccination policy;0;0;None;None;None;Government funded;No availability',
url_h3 = url_root_oxford + csv_h8 ## 'H8;Protection of elderly people;1;1;True;True;None;General;Recommended protections',

#### V[1-4] URLs
url_v1 = url_root_oxford + csv_h1 ## 'V1;Vaccine Prioritisation;-2;-2;None;None;None',
url_v2 = url_root_oxford + csv_h2 ## 'V2;Vaccine Availability;-2;-2;None;None;None',
url_v3 = url_root_oxford + csv_h3 ## 'V3;Vaccine Financial Support;-2;-2;None;None;None',
url_v4 = url_root_oxford + csv_h4 ## 'V4;Mandatory Vaccination;-2;-2;None;None;None']

### Vacinations Dataset URLs
url_root_vaccinations = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/'
url_v5  = url_root_vaccinations + csv_v5 ## '/vaccinations.csv'
url_v6 = url_root_vaccinations + csv_v6 ## 'vaccinations-by-age-group.csv'


#### DataFrame Index
!curl -L "{url_i1}" > {data_dir}{csv_i1}
!curl -L "{url_i2}" > {data_dir}{csv_i2}
!curl -L "{url_i3}" > {data_dir}{csv_i3}
!curl -L "{url_i4}" > {data_dir}{csv_i4}
!curl -L "{url_c1}" > {data_dir}{csv_c1}
!curl -L "{url_c2}" > {data_dir}{csv_c2}
!curl -L "{url_c3}" > {data_dir}{csv_c3}
!curl -L "{url_c4}" > {data_dir}{csv_c4}
!curl -L "{url_c5}" > {data_dir}{csv_c5}
!curl -L "{url_c6}" > {data_dir}{csv_c6}
!curl -L "{url_c7}" > {data_dir}{csv_c7}
!curl -L "{url_c8}" > {data_dir}{csv_c8}

df_raw_school_closing=pd.read_csv('../raw_data/c1m_school_closing.csv')
df_raw_workplace_closing=pd.read_csv('../raw_data/c2m_workplace_closing.csv')
df_raw_cancel_public_event=pd.read_csv('../raw_data/c3m_cancel_public_events.csv')
df_raw_restriction_on_gathering=pd.read_csv('../raw_data/c4m_restrictions_on_gatherings.csv')
df_raw_stay_at_home=pd.read_csv('../raw_data/c6m_stay_at_home_requirements.csv')
df_raw_international_travel=pd.read_csv('../raw_data/c6m_stay_at_home_requirements.csv')
df_raw_goverment_response=pd.read_csv('../raw_data/government_response_index_avg.csv')
df_raw_facial_covering=pd.read_csv('../raw_data/h6m_facial_coverings.csv')
df_raw_vacination_policy=pd.read_csv('../raw_data/h7_vaccination_policy.csv')



#### Vaccination
!curl -L "{url_vaccination}" > {data_dir}{vaccination_csv}
!curl -L "{url_ages}" > {data_dir}{ages_csv}

#### CSV target
!curl -L "{url_cases}" > {data_dir}{cases_csv}
!curl -L "{url_deaths}" > {data_dir}{deaths_csv}



### **Read CSV** and **Set raw dataframe**

In [ ]:
db_grid[csv_dict['containment_health_index_avg.csv']]

In [ ]:
df_raw_gov_response = pd.read_csv(data_dir + 'government_response_index_avg.csv')
df_raw_health = pd.read_csv(data_dir + 'containment_health_index_avg.csv')
df_raw_economic = pd.read_csv(data_dir + 'economic_support_index.csv')

#### Vaccination
df_raw_vaccination = pd.read_csv(data_dir + 'vaccinations.csv')
df_raw_ages = pd.read_csv(data_dir + 'vaccinations-by-age-group.csv')


#### Data Frame target
df_raw_cases = pd.read_csv(data_dir + 'confirmed_deaths.csv')
df_raw_deaths = pd.read_csv(data_dir + 'confirmed_deaths.csv')

#### Data multiple
# df_raw_confirmed_cases=pd.read_csv(url_cases)
# df_raw_death=pd.read_csv(data_dir + 'confirmed_deaths.csv')
df_raw_school_closing=pd.read_csv(data_dir + 'c1m_school_closing.csv')
df_raw_workplace_closing=pd.read_csv(data_dir + 'c2m_workplace_closing.csv')
df_raw_cancel_public_event=pd.read_csv(data_dir + 'c3m_cancel_public_events.csv')
df_raw_restriction_on_gathering=pd.read_csv(data_dir + 'c4m_restrictions_on_gatherings.csv')
df_raw_stay_at_home=pd.read_csv(data_dir + 'c6m_stay_at_home_requirements.csv')
df_raw_international_travel=pd.read_csv(data_dir + 'c6m_stay_at_home_requirements.csv')
df_raw_goverment_response=pd.read_csv(data_dir + 'government_response_index_avg.csv')
df_raw_facial_covering=pd.read_csv(data_dir + 'h6m_facial_coverings.csv')
df_raw_vacination_policy=pd.read_csv(data_dir + 'h7_vaccination_policy.csv')

In [ ]:
df_raw_cases.head()

In [ ]:
df_raw_cases.shape

## Exploratory Data Analyzis

In [ ]:
df_raw_cases.isna()

In [ ]:
# get VietNam country dataset
vn_data = df_raw_cases.loc[df_raw_cases['country_code'] == 'VNM'].copy()

vn_data.head()

In [ ]:
# %%time
# vn_data.profile_report()

### Time Series Analysis

In [ ]:
df_raw_cases.head()

In [ ]:
ts_cases = df_raw_cases.drop(columns=['country_name','region_code','region_name','jurisdiction','Unnamed: 0'])
ts_cases = ts_cases.groupby('country_code').agg('sum')
ts_cases.transpose()
ts_cases.columns.name = 'Dates'
ts_cases = ts_cases.fillna(0)
# ts_cases.index = pd.to_datetime(ts_cases.index)

In [ ]:
ts_cases = ts_cases.transpose()
ts_cases.head()

In [ ]:
ts_cases.shape

In [ ]:
vn_ts_cases = vn_data.drop(columns=['country_name','region_code','region_name','jurisdiction','Unnamed: 0'])
vn_ts_cases = vn_ts_cases.groupby('country_code').agg('sum')
vn_ts_cases.transpose()
vn_ts_cases.columns.name = 'Dates'
vn_ts_cases = vn_ts_cases.fillna(0)
# ts_cases.index = pd.to_datetime(ts_cases.index)

In [ ]:
vn_ts_cases

In [ ]:
vn_ts_cases = vn_ts_cases.transpose()

In [ ]:
vn_ts_cases

In [ ]:
country = 'France'

def country_output(country, indicator):
    
    # INDEX FEATURES
    country_index = df_gov_response.copy()
    country_index = country_index[[country]]
    country_index.index.name = country
    country_index.columns = ['Gov_response']
    country_index['Containment_and_health'] = df_health[[country]]
    country_index['Stringency'] = df_strigency[[country]]
    country_index['Economics_sup'] = df_economic[[country]]

    # POPULATION VACCINATED

    country_vaccination = df_vaccination.loc[df_vaccination['location']==country]
    country_vaccination = country_vaccination.fillna(method='ffill').drop(columns = 'location')
    country_vaccination.index.name = country
    country_vaccination = country_vaccination.fillna(0)

    # TARGET

    country_target = df_cases.copy()
    country_target = country_target[[country]]
    country_target.index.name = country
    country_target.columns = ['total_cases']
    country_target['new_cases'] = country_target - country_target.shift(1)
    country_target['total_deaths'] = df_deaths[[country]]
    country_target['new_deaths'] = df_deaths[[country]] - df_deaths[[country]].shift(1)

    country_target['new_cases'].loc[country_target['new_cases'] < 0] = 0
    country_target['new_deaths'].loc[country_target['new_deaths'] < 0] = 0

    # Days no update counter

    def non_uptade(country_target):

        counter = 0
        x = 1
        while country_target['total_deaths'][-x] == 0:
            counter += 1
            x += 1

        return counter

    counter = non_uptade(country_target)

    # Last Update Data

    country_index = country_index[:-counter]
    country_vaccination = country_vaccination[:-counter]
    country_target = country_target[:-counter]
    
    return country_index, country_vaccination, country_target

country_index, country_vaccination, country_target = country_output(country)

## Recurrent Neural Network (sequences data) modeling

### Samples/Sequences, Observations, Features

X.shape = (n_SEQUENCES, n_OBSERVATIONS, n_FEATURES)

y = RNN(X)

❗️ Notation $X_{i,j}^{t}$

 $_{i}$ is the sample/sequence
 
 $_{j}$ is the feature measured
 
 $^{t}$ is the time at which the observation is seen

In [ ]:
n_seq = 50 # ts_cases.shape[0] - 1 # nb of countries (samples)
n_obs = 15 # 15 days of training periiod (observations)
n_feat = 1 # 1feature: covid cases
n_pred = 1 # nb of days of prediction

X = np.array(ts).astype(np.float32)

y = np.array([y_a, y_b, y_c]).astype(np.float32)



### Prerequisites:

- **retrieve dataset** from Alberto

    - **clean dataset**: 
        
        - **drop first lines == 0** *(before Covid arrived)*
        
        - **check Nan**: 
- **strategy 1 country by country** sequences split as follow:

- **strategy 2 one sequence per country**:
    - **split X train, set** 
    - **Pad sequences**
    - **create one csv per country**

#### Create sequences (`X`,`y`)

def **subsample_sequence(df, length)** -> pd.DataFrame:

function that given a full dataframe `df`:
- Create a sub sequences df, with long length?

In [ ]:
def subsample_sequence(df, length) -> pd.DataFrame:
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length` (eg n_obs).
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    # How to split sequences? we could do it manually...
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    
    return df_sample

In [ ]:
# Test it 
assert subsample_sequence(vn_ts_cases, 10).shape  == (10, 1)
assert subsample_sequence(vn_ts_cases, 400).shape == (400, 1)
subsample_sequence(vn_ts_cases, 10).shape, subsample_sequence(vn_ts_cases, 400).shape

#### Pad `X` missing values with mean values

def **split_subsample_sequence(df,  length, sequence='VNM', df_mean=None)** -> tuple:

function that given a full dataframe `df`:
- Create a sub sequences df
- Stores the value of the covid deaths* (or cases) of the last day as your variable array `y`
- Stores all features of previous days as a variable `X`
- Returns (`X`, `y`)

* *'VNM_covd_deaths'* preference for death prediction, then switch on cases *'VNM_covid_cases'*

In [ ]:
# length of a sequence
len_seq = n_obs + n_pred
# Ex: 16 = 15 + 1
len_seq

In [ ]:
def split_subsample_sequence(df,  length, sequence='VNM', y_size=1) -> tuple:
    '''
    Create one single random (X_sample, y_sample)
    containing one sequence each of length `length`
    ToDo: Adapt the y size=-1'''
    # Trick to save time during potential recursive calls
    # if df_mean is None:
    #     df_mean = df.mean()
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length - y_size][sequence] # ['VNM'] ['VNM_covid_cases', 'VNM_covd_deaths'] 
    # Case y_sample is NaN: redraw !
    # if y_sample != y_sample: # A value is not equal to itself only for NaN
    #         X_sample, y_sample = split_subsample_sequence(df, length, df_mean) # Recursive call !!!
    #         return np.array(X_sample), np.array(y_sample)    
    X_sample = df_subsample[0:length - y_size]
    # Case X_sample has some NaNs
    # if X_sample.isna().sum().sum() !=0:
    #    X_sample = X_sample.fillna(compute_means(X_sample, df_mean))
    #    X_sample = X_sample.values

    return np.array(X_sample), np.array(y_sample)

In [ ]:
# Test it
(X_sample, y_sample) = split_subsample_sequence(vn_ts_cases, length=len_seq, y_size=n_pred)
assert X_sample.shape == (n_obs,n_feat)
assert y_sample.shape == ()
X_sample.shape, y_sample.shape

#### Generates an entire dataset of multiple subsamples with shape $(X, y)$

def **get_X_y(df, n_sequences, length)** -> tuple:

function to generates an entire dataset of multiple subsamples suitable for RNN, that is, $(X, y)$ of shape:

```python
X.shape = (n_sequences, length, n_features)
y.shape = (n_sequences, )
```

In [ ]:
def get_X_y(df, n_sequences, length) -> tuple:
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)

    return X, y

Generate your dataset $(X, y)$ of `50` sequences, each of `15` observations + the covid cases at the 16-th day to predict

n_seq = 50 # ts_cases.shape[0] - 1 # nb of countries (samples)
n_obs = 15 # 15 days of training periiod (observations)
n_feat = 1 # 1feature: covid cases
n_pred = 1 # nb of days of prediction
len_seq = 16 # length of a sequence (len_seq = n_obs + n_pred/ Ex: 16 = 15 + 1)

In [ ]:
X, y = get_X_y(vn_ts_cases, n_sequences=n_seq, length=len_seq)
print(X.shape)
print(y.shape)

In [ ]:
# Check your code below
assert X.shape == (50, 15, 1)
assert y.shape == (50, )
assert np.isnan(X).sum() == 0

### How to split sequences?

- randomly or

- manually

### Samples/Sequences, Observations, Features

### Split train

In [ ]:
# -1. Train splitting
# Let's keep the last 20% of the values out for testing purposes
train_size = 0.8 ## 80% of dataset to train
index = round(train_size * ts_cases.shape[0])

X_train = ts_cases.iloc[:index]
X_test = ts_cases.iloc[index:]

Let's not cross-validate in this challenge to start with 🤯 
- Separate `df` into `df_train` and `df_test` such that the first 80% of the dataframe is in the training, and the last 20% in the test set.
- Then generate (`X_train`, `y_train`) from `df_train` and (`X_test`, `y_test`) from `df_test`
- Ensure that `X_train.shape == (50, 15, 1)`

len_ = int(0.8*ts_cases.shape[0])
df_train = ts_cases[:len_] ; df_test = ts_cases[len_:]
df_train.shape, df_test.shape

In [ ]:
n_seq_test = n_seq // 3 ; n_seq_test

In [ ]:
X_train, y_train = get_X_y(df_train, n_seq, len_seq)
X_test, y_test = get_X_y(df_test, n_seq_test, len_seq)

In [ ]:
X_train.shape, X_test.shape

### Normalization layer

In [ ]:
# 0. The Normalization Layer
normalizer = Normalization()  # Instantiate a "normalizer" layer
normalizer.adapt(X_train) # "Fit" it on the train set

### RNN model architecture

In [ ]:
# 1. The Architecture
rnn_model = Sequential()
rnn_model.add(normalizer) # Using the Normalization layer to standardize the datapoints during the forward pass
rnn_model.add(SimpleRNN(units=20, activation='tanh', return_sequences=True))  ## , input_shape=(?,?))) without a Normalizer layer
rnn_model.add(SimpleRNN(units=20, activation='tanh'))  ## 
rnn_model.add(Dense(10, activation = 'relu')) ## add 1 or more 'relu' layers
# model.add(layers.Dropout(0.3)) ## if RNN model over-fit
rnn_model.add(Dense(n_pred, activation = 'linear'))

### Compile model with 'rmsprop'

In [ ]:
# 2. Compiling with 'rmsprop' rather than 'adam' (recommended)
rnn_model.compile(loss='mse',
              optimizer='rmsprop',
                 metrics='mape')  # Recommended optimizer for RNNs

In [ ]:
rnn_model.summary()

### Train model

In [ ]:
# 3. Training
es = EarlyStopping(monitor = 'val_loss',
                   patience = 10,
                   verbose = 0,
                   restore_best_weights = True)
# The fit
rnn_model.fit(X_train,
          y_train, 
          validation_split=0.1, # Auto split for validation data
              ## validation_data = (X_val, y_val), # To be created manually if needed
          batch_size = 16,
          epochs = 200,
          callbacks = [es],
          verbose=1)

### Evaluate model

In [ ]:
# 4. Evaluating
# The prediction (one per sequence/city)
y_pred = rnn_model.predict(X_test) 

In [ ]:
y_pred.shape

## Time Series Forecasting

In [ ]:
# Check your code below
assert y_pred.shape == (n_seq_test, n_pred)